In [1]:
import mindspore as ms
import mindspore.ops as ops

x = ops.arange(4.0)
x

Tensor(shape=[4], dtype=Float32, value= [ 0.00000000e+00,  1.00000000e+00,  2.00000000e+00,  3.00000000e+00])

在我们$y$计算关于$\\{x}$的梯度之前，我们需要介绍一下MindSpore的自动微分实现方式

MindSpore现有版本同时支持静态图和动态图，即GRAPH_MODE和PYNATIVE_MODE，为了将二者统一，整体进行自动微分的方式为静态图方式，即： 将函数视为一个完整的计算图，先进行编译，后执行。因此不像Pytorch一样将梯度grad直接绑定在Tensor上，而是整图运算后，再通过取梯度的算子进行梯度的提取。（动态图模式也并非纯Python执行，二者均采取静态图的策略）因此，和Pytorch有如下差异：

   1.想要自动微分的函数需要显式注册为function

   2.需要通过`ops.GradOperation`算子来获取梯度

现在计算$y$

In [2]:
import mindspore.numpy as mnp

def forward(x):
    return 2 * mnp.dot(x, x)

y = forward(x)
y

Tensor(shape=[], dtype=Float32, value= 28)

通过调用`ops.GradOperation`算子来自动计算y关于x每个分量的梯度

In [3]:
grad_all = ops.GradOperation(get_all=True)
x_grad = grad_all(forward)(x)[0]
x_grad

Tensor(shape=[4], dtype=Float32, value= [ 0.00000000e+00,  4.00000000e+00,  8.00000000e+00,  1.20000000e+01])

In [4]:
x_grad == 4 * x

Tensor(shape=[4], dtype=Bool, value= [ True,  True,  True,  True])

现在让我们计算x的另一个函数

In [5]:
def forward(x):
    return x.sum()

x_grad = grad_all(forward)(x)[0]
x_grad

Tensor(shape=[4], dtype=Float32, value= [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00])

In [6]:
def forward(x):
    y = x * x
    return y.sum()

x_grad = grad_all(forward)(x)[0]
x_grad

Tensor(shape=[4], dtype=Float32, value= [ 0.00000000e+00,  2.00000000e+00,  4.00000000e+00,  6.00000000e+00])

In [7]:
def forward(x):
    y = x * x
    u = ops.stop_gradient(y)
    z = u * x
    return z, u

z, u = forward(x)
x_grad = grad_all(forward)(x)[0]
x_grad == u

Tensor(shape=[4], dtype=Bool, value= [ True,  True,  True,  True])

In [8]:
def forward(x):
    y = x * x
    return y.sum()
x_grad = grad_all(forward)(x)[0]
x_grad == 2 * x

Tensor(shape=[4], dtype=Bool, value= [ True,  True,  True,  True])

In [9]:
def f(a):
    b = a * 2
    while ops.norm(b, axis=0) < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [10]:
a = ops.randn(())
d = f(a)
a_grad = grad_all(f)(a)[0]

In [11]:
a_grad == d / a

Tensor(shape=[], dtype=Bool, value= True)